<a href="https://colab.research.google.com/github/AdamKhalil27/DataBase/blob/main/DB_Labb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00


In [1]:
import sqlite3
import pandas as pd
%load_ext sql


# Connect to SQLite database (or create it if it doesn’t exist)
conn = sqlite3.connect("lab9.db")
c = conn.cursor()


# Define SQL commands
commands = [
    # Create 'students' table
    """
    CREATE TABLE IF NOT EXISTS students (
        student_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        major TEXT
    );
    """,

    # Insert data into 'students' table
    """
    INSERT INTO students (first_name, last_name, major) VALUES
        ('John', 'Doe', 'Computer Science'),
        ('Jane', 'Doe', 'Mathematics'),
        ('Mike', 'Jordan', 'Physics');
    """,

    # Create 'departments' table
    """
    CREATE TABLE IF NOT EXISTS departments (
        department_id INTEGER PRIMARY KEY,
        department_name TEXT
    );
    """,

    # Insert data into 'departments' table
    """
    INSERT INTO departments (department_name) VALUES
        ('Computer Science'),
        ('Mathematics'),
        ('Physics'),
        ('Chemistry');
    """,

    # Create 'instructors' table
    """
    CREATE TABLE IF NOT EXISTS instructors (
        instructor_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        department_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id)
    );
    """,

    # Insert data into 'instructors' table
    """
    INSERT INTO instructors (first_name, last_name, department_id) VALUES
        ('Robert', 'Smith', 1),
        ('Susan', 'Johnson', 2),
        ('Michael', 'Brown', 3),
        ('Elizabeth', 'Taylor', 4);
    """,

    # Create 'courses' table
    """
    CREATE TABLE IF NOT EXISTS courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT,
        department_id INTEGER,
        instructor_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id),
        FOREIGN KEY (instructor_id) REFERENCES instructors(instructor_id)
    );
    """,

    # Insert data into 'courses' table
    """
    INSERT INTO courses (course_name, department_id, instructor_id) VALUES
        ('Database Systems', 1, 1),
        ('Data Structures', 1, 2),
        ('Linear Algebra', 2, 3),
        ('Quantum Mechanics', 3, 4);
    """,

    # Create 'enrollments' table
    """
    CREATE TABLE IF NOT EXISTS enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        course_id INTEGER,
        FOREIGN KEY (student_id) REFERENCES students(student_id),
        FOREIGN KEY (course_id) REFERENCES courses(course_id)
    );
    """,

    # Insert data into 'enrollments' table
    """
    INSERT INTO enrollments (student_id, course_id) VALUES
        (1, 1),
        (1, 2),
        (2, 3),
        (3, 4);
    """
]

# Execute SQL commands
for command in commands:
    c.execute(command)

# Commit changes and close the connection
conn.commit()
conn.close()

In [4]:
# Reconnect to the SQLite database
conn = sqlite3.connect("lab9.db")
# Execute INNER JOIN query and visualize the result
inner_join_query = """
SELECT students.first_name AS student_name , courses.course_name FROM students
INNER JOIN enrollments ON students.student_id = enrollments. student_id
INNER JOIN courses ON enrollments.course_id = courses.course_id; """
inner_join_df = pd.read_sql_query(inner_join_query , conn)
inner_join_df

,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [5]:
# Execute LEFT JOIN query and visualize the result
left_join_query = """
SELECT students.first_name AS student_name , courses.course_name
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.
student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id;
"""
left_join_df = pd.read_sql_query(left_join_query , conn)
left_join_df

,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [6]:
# Reconnect to the SQLite database
conn = sqlite3.connect("lab9.db")
# Execute FULL JOIN query (emulated) and visualize the result
full_join_query_1 = """
WITH RECURSIVE full_join AS (
SELECT students.first_name AS student_name , courses.course_name
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.
student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id
UNION ALL
SELECT students.first_name AS student_name , courses.course_name
FROM courses
LEFT JOIN enrollments ON courses.course_id = enrollments.
course_id
LEFT JOIN students ON enrollments.student_id = students.
student_id
WHERE students.student_id IS NULL
)
SELECT * FROM full_join;
"""
full_join_df_1 = pd.read_sql_query(full_join_query_1 , conn)
full_join_df_1

,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [ ]:

# Reconnect to the SQLite database
conn = sqlite3.connect("lab9.db")
# Execute Subquery and visualize the result
subquery_example_1 = """
SELECT first_name , last_name
FROM students
WHERE student_id IN ( SELECT student_id
FROM enrollments WHERE course_id IN (
SELECT course_id FROM courses
WHERE course_name = ’Database Systems’ )
); """
subquery_df_1 = pd.read_sql_query(subquery_example_1 , conn)
subquery_df_1


In [ ]:
# Execute Subquery and visualize the result
subquery_example_2 = """
SELECT course_name FROM courses
WHERE department_id = (
SELECT department_id FROM departments
WHERE department_name ='Computer Science' );
"""
subquery_df_2 = pd.read_sql_query(subquery_example_2 , conn)
subquery_df_2

In [8]:
# Reconnect to the SQLite database
conn = sqlite3.connect("lab9.db")
# Execute Aggregate Function query and visualize the result
agg_function_example_1 = """
SELECT departments.department_name , COUNT(DISTINCT instructors.
instructor_id) AS instructor_count FROM departments
LEFT JOIN instructors ON departments.department_id = instructors.
department_id
GROUP BY departments.department_name;
"""
agg_df_1 = pd.read_sql_query(agg_function_example_1 , conn)
agg_df_1


,department_name,instructor_count
0,Chemistry,1
1,Computer Science,1
2,Mathematics,1
3,Physics,1


In [9]:

# Execute Aggregate Function query and visualize the result
agg_function_example_2 = """
SELECT courses.course_name , COUNT(DISTINCT enrollments.student_id)
AS student_count FROM courses
LEFT JOIN enrollments ON courses.course_id = enrollments.course_id GROUP BY courses.course_name
HAVING student_count > 0; """
agg_df_2 = pd.read_sql_query(agg_function_example_2 , conn)
agg_df_2

,course_name,student_count
0,Data Structures,1
1,Database Systems,1
2,Linear Algebra,1
3,Quantum Mechanics,1


In [10]:

conn = sqlite3.connect('lab9.db')
c = conn.cursor()

count_courses_query = """
SELECT d.department_name, COUNT(c.course_id) AS course_count
FROM departments d
LEFT JOIN courses c ON d.department_id = c.department_id
GROUP BY d.department_name
"""

try:
    c.execute(count_courses_query)

    results = c.fetchall()

    for row in results:
        department_name, course_count = row
        print(f"Department: {department_name}, Number of Courses: {course_count}")
finally:
    # Close the cursor and the database connection
    c.close()
    conn.close()

Department: Chemistry, Number of Courses: 0
Department: Computer Science, Number of Courses: 2
Department: Mathematics, Number of Courses: 1
Department: Physics, Number of Courses: 1


In [11]:

conn = sqlite3.connect('lab9.db')
c = conn.cursor()

count_instructors_query = """
SELECT d.department_name, COUNT(i.instructor_id) AS instructor_count
FROM departments d
LEFT JOIN instructors i ON d.department_id = i.department_id
GROUP BY d.department_name
"""

try:
    c.execute(count_instructors_query)

    results = c.fetchall()

    for row in results:
        department_name, instructor_count = row
        print(f"Department: {department_name}, Number of Instructors: {instructor_count}")
finally:
    c.close()
    conn.close()

Department: Chemistry, Number of Instructors: 1
Department: Computer Science, Number of Instructors: 1
Department: Mathematics, Number of Instructors: 1
Department: Physics, Number of Instructors: 1


In [12]:

conn = sqlite3.connect('lab9.db')
c = conn.cursor()

average_students_per_course_query = """
SELECT d.department_name, AVG(enrollment_count) AS average_students_per_course
FROM departments d
LEFT JOIN (
    SELECT c.department_id, COUNT(e.student_id) AS enrollment_count
    FROM courses c
    LEFT JOIN enrollments e ON c.course_id = e.course_id
    GROUP BY c.department_id
) subquery
ON d.department_id = subquery.department_id
GROUP BY d.department_name
"""

try:
    c.execute(average_students_per_course_query)

    results = c.fetchall()

    for row in results:
        department_name, average_students_per_course = row
        if average_students_per_course is not None:
            print(f"Department: {department_name}, Average Students Per Course: {average_students_per_course:.2f}")
        else:
            print(f"Department: {department_name}, No Courses or Enrollments")
finally:
    c.close()
    conn.close()

Department: Chemistry, No Courses or Enrollments
Department: Computer Science, Average Students Per Course: 2.00
Department: Mathematics, Average Students Per Course: 1.00
Department: Physics, Average Students Per Course: 1.00
